In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

Load Google Drive Content

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/housing_price_prediction/hpp_dataset.csv")

In [ ]:
df.head(10)

,id,url,region,region_url,price,type,sqfeet,beds,baths,cats_allowed,...,wheelchair_access,electric_vehicle_charge,comes_furnished,laundry_options,parking_options,image_url,description,lat,long,state
0,7039061606,https://bham.craigslist.org/apa/d/birmingham-h...,birmingham,https://bham.craigslist.org,1195,apartment,1908,3,2.0,1,...,0,0,0,laundry on site,street parking,https://images.craigslist.org/00L0L_80pNkyDeG0...,Apartments In Birmingham AL Welcome to 100 Inv...,33.4226,-86.7065,al
1,7041970863,https://bham.craigslist.org/apa/d/birmingham-w...,birmingham,https://bham.craigslist.org,1120,apartment,1319,3,2.0,1,...,0,0,0,laundry on site,off-street parking,https://images.craigslist.org/00707_uRrY9CsNMC...,Find Your Way to Haven Apartment Homes Come ho...,33.3755,-86.8045,al
2,7041966914,https://bham.craigslist.org/apa/d/birmingham-g...,birmingham,https://bham.craigslist.org,825,apartment,1133,1,1.5,1,...,0,0,0,laundry on site,street parking,https://images.craigslist.org/00h0h_b7Bdj1NLBi...,Apartments In Birmingham AL Welcome to 100 Inv...,33.4226,-86.7065,al
3,7041966936,https://bham.craigslist.org/apa/d/birmingham-f...,birmingham,https://bham.craigslist.org,800,apartment,927,1,1.0,1,...,0,0,0,laundry on site,street parking,https://images.craigslist.org/00808_6ghZ8tSRQs...,Apartments In Birmingham AL Welcome to 100 Inv...,33.4226,-86.7065,al
4,7041966888,https://bham.craigslist.org/apa/d/birmingham-2...,birmingham,https://bham.craigslist.org,785,apartment,1047,2,1.0,1,...,0,0,0,laundry on site,street parking,https://images.craigslist.org/00y0y_21c0FOvUXm...,Apartments In Birmingham AL Welcome to 100 Inv...,33.4226,-86.7065,al
5,7041966868,https://bham.craigslist.org/apa/d/birmingham-s...,birmingham,https://bham.craigslist.org,900,apartment,1298,2,2.0,1,...,0,0,0,laundry on site,street parking,https://images.craigslist.org/00606_g79izH5xxk...,Apartments In Birmingham AL Welcome to 100 Inv...,33.4226,-86.7065,al
6,7041966702,https://bham.craigslist.org/apa/d/birmingham-2...,birmingham,https://bham.craigslist.org,925,apartment,1350,2,2.0,1,...,0,0,0,laundry on site,street parking,https://images.craigslist.org/00505_cAu36QXQV4...,Apartments In Birmingham AL Welcome to 100 Inv...,33.4226,-86.7065,al
7,7041964186,https://bham.craigslist.org/apa/d/calera-st-av...,birmingham,https://bham.craigslist.org,1085,house,2308,3,2.0,1,...,0,0,0,w/d hookups,NaN,https://images.craigslist.org/00D0D_g4BE3jPMBr...,"This is a NEW and LARGE 3BR, 2BTH renovation i...",33.0969,-86.7601,al
8,7041960353,https://bham.craigslist.org/apa/d/birmingham-n...,birmingham,https://bham.craigslist.org,1269,apartment,1156,3,2.0,1,...,0,0,0,w/d in unit,NaN,https://images.craigslist.org/00a0a_lTBiptw032...,Perfect Location and a Perfect Price!!! Come ...,33.4237,-86.8015,al
9,7041959413,https://bham.craigslist.org/apa/d/birmingham-a...,birmingham,https://bham.craigslist.org,799,apartment,703,1,1.0,1,...,0,0,0,w/d in unit,NaN,https://images.craigslist.org/00707_9dHoTSYoQn...,HWY 31 and I-65 access for easy commute! Be Do...,33.4237,-86.8015,al


# Craigslist Rental Listings Dataset Description
This dataset contains information on rental listings from Craigslist, covering various regions. The dataset is structured with the following attributes:

Id: The unique identifier for each listing.

url: The URL link to the listing.

region: The Craigslist region where the listing is located.

region_url: The URL link to the region.

price: The monthly rent for the listing (Target Column).

type: The type of housing (e.g., apartment, house, etc.).

sqfeet: The total square footage of the property.

beds: The number of bedrooms.

baths: The number of bathrooms.

cats_allowed: Boolean indicating if cats are allowed (1 = yes, 0 = no).

dogs_allowed: Boolean indicating if dogs are allowed.

smoking_allowed: Boolean indicating if smoking is allowed.

wheelchair_access: Boolean indicating if the property has wheelchair access.

electric_vehicle_charge: Boolean indicating if the property has an electric vehicle charger.

comes_furnished: Boolean indicating if the property comes furnished.

laundry_options: Available laundry options.

parking_options: Available parking options.

image_url: The URL link to images of the property.

description: Description provided by the poster.

lat: Latitude coordinates of the listing.

long: Longitude coordinates of the listing.

state: The state where the listing is located.

Clean dataset

In [ ]:
# remove columns with less or no predictive power
df = df.drop(columns=['id','url','region','region_url','image_url','description', 'lat', 'long'])
print('After dropping unwanted columns')
print(df.head(1))

After dropping unwanted columns
   price       type  sqfeet  beds  baths  cats_allowed  dogs_allowed  \
0   1195  apartment    1908     3    2.0             1             1   

   smoking_allowed  wheelchair_access  electric_vehicle_charge  \
0                1                  0                        0   

   comes_furnished  laundry_options parking_options state  
0                0  laundry on site  street parking    al  


In [ ]:
# view column types
df.dtypes

region                      object
price                        int64
type                        object
sqfeet                       int64
beds                         int64
baths                      float64
cats_allowed                 int64
dogs_allowed                 int64
smoking_allowed              int64
wheelchair_access            int64
electric_vehicle_charge      int64
comes_furnished              int64
laundry_options             object
parking_options             object
state                       object
dtype: object

In [ ]:
df.describe()

,price,sqfeet,beds,baths,cats_allowed,dogs_allowed,smoking_allowed,wheelchair_access,electric_vehicle_charge,comes_furnished
count,2.651900e+05,2.651900e+05,265190.000000,265190.000000,265190.000000,265190.000000,265190.000000,265190.000000,265190.000000,265190.000000
mean,1.227285e+04,1.093678e+03,1.912414,1.483468,0.716822,0.696550,0.733896,0.078759,0.014360,0.048644
std,5.376352e+06,2.306888e+04,3.691900,0.630208,0.450543,0.459749,0.441920,0.269362,0.118968,0.215124
min,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8.170000e+02,7.520000e+02,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.060000e+03,9.500000e+02,2.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
75%,1.450000e+03,1.156000e+03,2.000000,2.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
max,2.768307e+09,8.388607e+06,1100.000000,75.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


from matplotlib import pyplot as plt
_df_0['price'].plot(kind='hist', bins=20, title='price')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['sqfeet'].plot(kind='hist', bins=20, title='sqfeet')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['beds'].plot(kind='hist', bins=20, title='beds')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3['baths'].plot(kind='hist', bins=20, title='baths')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
_df_7.plot(kind='scatter', x='baths', y='cats_allowed', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_8['price'].plot(kind='line', figsize=(8, 4), title='price')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_9['sqfeet'].plot(kind='line', figsize=(8, 4), title='sqfeet')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_10['beds'].plot(kind='line', figsize=(8, 4), title='beds')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_11['baths'].plot(kind='line', figsize=(8, 4), title='baths')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
# Check for empty values
missing_values_per_column = df.isnull().sum()
print(missing_values_per_column)

price                          0
type                           0
sqfeet                         0
beds                           0
baths                          0
cats_allowed                   0
dogs_allowed                   0
smoking_allowed                0
wheelchair_access              0
electric_vehicle_charge        0
comes_furnished                0
laundry_options            54311
parking_options            95135
state                          1
dtype: int64


Laundry options and parking options have considerable amount of null values. Predictive power of laundry options is less, hence we will drop the column laundry_options. However parking options has a higher effect on housing price, hence we will drop rows with missing values in column parking_options instead of dropping the entire column.

In [ ]:
#drop laundry options
df = df.drop(columns=['laundry_options'])

In [ ]:
#drop rows with empty values
row_count_before_dropna = len(df.index)
df.dropna(inplace = True)
row_count_after_dropna = len(df.index)
print(row_count_before_dropna - row_count_after_dropna, ' rows dropped!')

95136  rows dropped!


In [ ]:
# min max scaling
columns_to_scale = ['sqfeet', 'beds', 'baths']
scaler = MinMaxScaler()

scaled_columns = scaler.fit_transform(df[columns_to_scale])
scaled_df = pd.DataFrame(scaled_columns, columns=columns_to_scale, index=df.index)
df = pd.concat([df.drop(columns=columns_to_scale), scaled_df], axis=1)

In [ ]:
print('After min max scaling')
df.describe()

After min max scaling


,price,cats_allowed,dogs_allowed,smoking_allowed,wheelchair_access,electric_vehicle_charge,comes_furnished,sqfeet,beds,baths
count,1.700540e+05,170054.000000,170054.000000,170054.000000,170054.000000,170054.000000,170054.000000,170054.000000,170054.000000,170054.000000
mean,1.833022e+04,0.763175,0.738030,0.652263,0.100186,0.018859,0.060304,0.000137,0.001758,0.019682
std,6.713660e+06,0.425135,0.439708,0.476253,0.300248,0.136026,0.238051,0.003434,0.004149,0.008619
min,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8.200000e+02,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000089,0.000909,0.013333
50%,1.079000e+03,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000113,0.001818,0.013333
75%,1.500000e+03,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000138,0.001818,0.026667
max,2.768307e+09,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
#one hot encoding
df = pd.get_dummies(df, columns = ['type','parking_options','state'])

In [ ]:
print('After one hot encoding ...')
df.head(3)

After one hot encoding ...


,price,cats_allowed,dogs_allowed,smoking_allowed,wheelchair_access,electric_vehicle_charge,comes_furnished,sqfeet,beds,baths,...,state_nd,state_ne,state_nh,state_nj,state_nm,state_nv,state_ny,state_oh,state_ok,state_or
0,1195,1,1,1,0,0,0,0.000227,0.002727,0.026667,...,False,False,False,False,False,False,False,False,False,False
1,1120,1,1,1,0,0,0,0.000157,0.002727,0.026667,...,False,False,False,False,False,False,False,False,False,False
2,825,1,1,1,0,0,0,0.000135,0.000909,0.020000,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
print('Dimensions of final dataset after cleaning', df.shape)

Dimensions of final dataset after cleaning (170054, 67)


In [ ]:
X = df.drop(columns=['price'])
Y = df['price']

In [ ]:
X.shape

(170054, 66)

In [ ]:
Y.shape

(170054,)

In [ ]:
#Splitting the datset into training and testing
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20, random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
from sklearn.metrics import mean_squared_error
linear = LinearRegression()
linear.fit(X_train, Y_train)
pred=linear.predict(X_test)

linear_rsq = (linear.score(X_test,Y_test))*100
linear_mse = mean_squared_error(pred, Y_test)

In [ ]:
lasso = Lasso(alpha = 0.1)
lasso.fit(X_train, Y_train)
lasso_pred=lasso.predict(X_test)

lasso_rsq = (lasso.score(X_test,Y_test))*100
lasso_mse = mean_squared_error(lasso_pred, Y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.844e+18, tolerance: 7.665e+14
  model = cd_fast.enet_coordinate_descent(


In [ ]:
dec = DecisionTreeRegressor()
dec.fit(X_train, Y_train)
dec_pred=dec.predict(X_test)

dec_rsq = (dec.score(X_test,Y_test))*100
dec_mse = mean_squared_error(dec_pred, Y_test)

In [ ]:
el = ElasticNet()
el.fit(X_train, Y_train)
el_pred=el.predict(X_test)

el_rsq = (el.score(X_test,Y_test))*100
el_mse = mean_squared_error(el_pred, Y_test)

In [ ]:
kn = KNeighborsRegressor()
kn.fit(X_train, Y_train)
kn_pred=kn.predict(X_test)

kn_rsq = (kn.score(X_test,Y_test))*100
kn_mse = mean_squared_error(kn_pred, Y_test)

In [31]:
list_reg=['Linear','Lasso','ElasticNet','Decision Tree','KNeighbors']
data = {'Regressor': list_reg, 'R Score': [linear_rsq, lasso_rsq, el_rsq,dec_rsq,kn_rsq], 'MSE':[linear_mse, lasso_mse, el_mse,dec_mse,kn_mse]}
reg_df=pd.DataFrame(data)
reg_df

,Regressor,R Score,MSE
0,Linear,38.095914,0.306778
1,Lasso,-0.212319,0.496622
2,ElasticNet,-0.212319,0.496622
3,Decision Tree,36.596259,0.314210
4,KNeighbors,31.219332,0.340856


from matplotlib import pyplot as plt
reg_df['R Score'].plot(kind='hist', bins=20, title='R Score')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
reg_df['MSE'].plot(kind='hist', bins=20, title='MSE')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
reg_df.groupby('Regressor').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
reg_df.plot(kind='scatter', x='R Score', y='MSE', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
reg_df['MSE'].plot(kind='line', figsize=(8, 4), title='MSE')
plt.gca().spines[['top', 'right']].set_visible(False)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(reg_df['Regressor'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(reg_df, x='R Score', y='Regressor', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(reg_df['Regressor'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(reg_df, x='MSE', y='Regressor', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)